In [1]:
import os
import json
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [2]:
assistant_id = "asst_bLvSlz40u6JfYR30LpRDqLTe"

assistant = client.beta.assistants.retrieve(assistant_id)
print(assistant)

Assistant(id='asst_bLvSlz40u6JfYR30LpRDqLTe', created_at=1723142192, description=None, file_ids=[], instructions='You are an AI Agent who helps EMS (Emergency Medical Service) personnel create patient charts. You only provide concise replies that are polite and professional. You can ask question to guide user to fill our the patient charts. You answer questions truthfully based on official information, with consideration to context provided below on what information is needed.Do not answer questions that are not related to patient charts and respond with I can only help with any EMS patient chart questions you may have.If you do not know the answer to a question, respond by saying “I do not know the answer to your question”You are given various functions that provide additional the data needed to answer questions.## Information Need For Patient Chart: - Why was EMS dispatch, mechanism of injury and chief complain - Patient information (age, gender) - Scene information for example house

In [3]:
thread = client.beta.threads.create()
print(thread.id)

thread_npCvR2qAFHlDOVtXZXh7MbfG


In [4]:
# Add a user question to the thread
msg = "Can you help me with a patient chart?"

#msg = "email me a summary of this conversation to ddiaz@live.com thread_id: thread_4i0QVGUqIbnjd5SmNgXOCHib"

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=msg
)

thread_messages = client.beta.threads.messages.list(thread.id)

In [5]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [6]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


In [ ]:
from functions.email import send_summary

#send_summary("ddiaz@live.com", thread.id)

In [ ]:
from functions.patient import get_patient_information
from functions.patient import get_patient_chart
from functions.email import send_report

import time

max_steps = 10 
sleep = .5

#add the available functions here   
available_functions = {"get_patient_information": get_patient_information, "get_patient_chart": get_patient_chart, "send_summary": send_summary}

try:
    cnt = 0
    while cnt < max_steps:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)        
        cnt += 1
        if run.status == "requires_action":
            tool_responses = []
            if (
                run.required_action.type == "submit_tool_outputs"
                and run.required_action.submit_tool_outputs.tool_calls is not None
            ):
                tool_calls = run.required_action.submit_tool_outputs.tool_calls

                for call in tool_calls:
                    if call.type == "function":
                        if call.function.name not in available_functions:
                            raise Exception("Function requested by the model does not exist")
                        function_to_call = available_functions[call.function.name]
                        print(call.function.arguments)
                        print(call.function.name)
                        tool_response = function_to_call(**json.loads(call.function.arguments))
                        tool_responses.append({"tool_call_id": call.id, "output": tool_response})

            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_responses
            )

            print(run.status)

        if run.status == "failed":
            print("Run failed.")
            break
        if run.status == "completed":
            break
        time.sleep(sleep)

except Exception as e:
    print(e)



In [7]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))
print(run.status)

{
  "data": [
    {
      "id": "msg_qsP0ZBm8RBHK6GPdPxkmXNiH",
      "assistant_id": "asst_bLvSlz40u6JfYR30LpRDqLTe",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Certainly! I can help you with creating a patient chart. Let's start with the necessary details:\n\n1. Why was EMS dispatched, mechanism of injury, and chief complaint?\n2. Patient information (age, gender)?\n3. Scene information (e.g., house, apartment, alley, car accident, etc.)?\n4. Symptoms?\n5. Patient vitals (heart rate, blood pressure, respiratory rate, blood oxygen, body temperature)?\n6. Allergies?\n7. Medical history?\n8. Medications?\n9. Last oral intake?\n\nPlease provide the information so I can assist you further."
          },
          "type": "text"
        }
      ],
      "created_at": 1726691154,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_t2rTNmW8mzHFCExJZu7C1yso",
  